In [7]:
import json
import os

import pandas as pd
from web3 import Web3
from web3.middleware import geth_poa_middleware

from alpha_homora_v2.util import ContractInstanceFunc, cov_from, get_web3_provider
from alpha_homora_v2.oracle import GetPriceFunc
from alpha_homora_v2.resources.abi_reference import *


In [9]:
# Setup Web3 Provider:
NETWORK_RPC_URL = "https://api.avax.network/ext/bc/C/rpc"

w3_provider = get_web3_provider(NETWORK_RPC_URL)

In [11]:
# Setting

position_id = 10936

# real-time price
avax_price = GetPriceFunc('AVAX')
eth_price = GetPriceFunc('ETH')
png_price = GetPriceFunc('PNG')


In [12]:
bank_instance = ContractInstanceFunc(w3_provider, *HomoraBank_ABI)
TraderJoeSpellV1_ins = ContractInstanceFunc(w3_provider, *TraderJoeSpellV1_ABI)
PangolinSpellV2_instance = ContractInstanceFunc(w3_provider, *PangolinSpellV2_ABI)
AggregatorOracle_ins = ContractInstanceFunc(w3_provider, *AggregatorOracle_ABI)
WERC20_ins = ContractInstanceFunc(w3_provider, *WERC20_ABI)
WMasterchefJoeV2_instance = ContractInstanceFunc(w3_provider, *WMasterchefJoeV2_ABI)
coll_instance = ContractInstanceFunc(w3_provider, *WMiniChefPNG_ABI)
pool_instance = ContractInstanceFunc(w3_provider, *PangolinLiquidity_ABI)
usdce_instance = ContractInstanceFunc(w3_provider, *USDCe_ABI)
png_minishefv2_ins = ContractInstanceFunc(w3_provider, *MiniChefV2_ABI)

In [13]:
# Debt

position_debt_output = bank_instance.functions.getPositionDebts(position_id).call()
position_info_output = bank_instance.functions.getPositionInfo(position_id).call()

# print(position_debt_output)
# print(position_info_output)

position_debt = position_debt_output[1][0]/1e18 * avax_price
print("Position Debt:", position_debt)
collateral_size = position_info_output[3]

Position Debt: 591.5155678650119


In [14]:
# Position Value 

r0,r1,_ = pool_instance.functions.getReserves().call()
supply = pool_instance.functions.totalSupply().call()

token0_amount = r0*collateral_size//supply/1e6
token1_amount = r1*collateral_size//supply/1e18

position_value = token0_amount + token1_amount*avax_price
print("Position Value:", position_value)

Position Value: 1644.9485227228456


In [15]:
position_info_output

['0x3F53B4e92f0A321723b69b3D9efE42b90d3248Da',
 '0x8DF47fc33DF77Ae0526cDAC4A0CA89739EF9f1Cc',
 5300541194335152988749892502228755547482451690626931611734137475369893570,
 149150448042682]

In [ ]:
# Reward

pid, entryRewardPerShare = coll_instance.functions.decodeId(position_info_output[2]).call()
accRewardPerShare = png_minishefv2_ins.functions.poolInfo(pid).call()[0]
# rewardAmount in PNG token
rewardAmount = collateral_size * cov_from(accRewardPerShare - entryRewardPerShare) /10e11
reward_value = rewardAmount*png_price

In [ ]:
accRewardPerShare

In [ ]:
# Debt Ratio

collateral_credit = bank_instance.functions.getCollateralETHValue(position_id).call()
borrow_credit = bank_instance.functions.getBorrowETHValue(position_id).call()

debt_ratio = borrow_credit/collateral_credit

In [ ]:
# Output

dict1 = {
    "Position Debt":position_debt, 
    "Position Value": position_value, 
    "Debt Ratio": debt_ratio,
    "Reward Amount": rewardAmount,
    "Reward Value": reward_value}

df_monitor = pd.DataFrame(dict1, index=[position_id])

In [ ]:
display(df_monitor)
df_monitor.to_csv(f'{os.path.abspath(os.path.dirname(os.getcwd()))}\\Result\\Position {position_id} Info.csv')